In [350]:
import nltk 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize, sent_tokenize 



In [351]:
import re
def is_measure_word(word):
    quantifier1 = ['bowl','dish','pound','piece','can','box','bag','carton','jar','loaf','slice','package','dash','cube','pack','head','ear','kernel','gain','grain','stalk','spear','clove']
    quantifier2 = ['teaspoon','tablespoon','glass','cup','pint','quart','half gallon','gallon','tank','jug','bottle','keg','shot','drop']
    quantifier3 = ['bar','tube','container','stick','roll','ball','spool','skein','yard','meter','foot','piece','pad','roll','stick','pair']
    for q in quantifier1:
        if re.search(q,word):
            return True
    for q in quantifier2:
        if re.search(q,word):
            return True
    for q in quantifier3:
        if re.search(q,word):
            return True
        
    return False
# print(is_measure_word('jars'))
def tokenize_ingredient(text):
    ingredient = {}
    tokenized = sent_tokenize(text) 
    quantity = ''
    ingredient_name = ''
    measurement = ''
    preparation = ''
    measure_word = False
    ignore = False
    for token in tokenized: 
        wordsList = nltk.word_tokenize(token) 
        tagged = nltk.pos_tag(wordsList) 
        for tag in tagged:
            if tag[1] == 'CD' and not ignore:
                quantity += tag[0]+' '
                measure_word = True
            elif tag[0]== '(':
                m = re.search(r"\(([0-9]* [A-Za-z0-9_]+)\)", s)
                quantity += '(' +m.group(1)+') '
                ignore = True
            elif tag[0] == ')':
                ignore = False
                measure_word = True
            elif (tag[1] == 'NN' or tag[1]=='NNS' or tag[1]=='VBZ') and not ignore:
                word =  tag[0]
#                 print(word)
                if measure_word:
                    if is_measure_word(word):
                        measurement += word
                    else:
                        ingredient_name += word+' '
                    measure_word = False
                else:
                    ingredient_name += word+' '
            elif tag[0]==',':
                preparation = token.split(',')[1]
                break
            elif not ignore:
                ingredient_name+=tag[0]+' '
#         print(tagged) 
    ingredient['ingredient_name']=ingredient_name
    ingredient['quantity'] = quantity
    ingredient['measurement'] = measurement
    ingredient['preparation'] = preparation
#     print(ingredient)
    return ingredient

    

tokenize_ingredient("12 whole wheat lasagna noodles")
tokenize_ingredient('1 1/2 (25 ounce) jars tomato-basil pasta sauce')
tokenize_ingredient(' 2 cloves garlic, chopped')
tokenize_ingredient(' 1 (60 ounce) package cottage cheese')
tokenize_ingredient('  1 teaspoon dried oregano, or to taste')


{'ingredient_name': 'dried oregano ',
 'quantity': '1 ',
 'measurement': 'teaspoon',
 'preparation': ' or to taste'}

In [352]:
import re
    
def get_ingredient(word, ingredients):
    bad_words = ['(',')','[',']','{','}']
    for ingredient in  ingredients:
        if word in bad_words:
            continue
#         print('word',word,'ingredient',ingredient)
        if re.search(word, ingredient):
            return ingredient
    return False

# print(get_ingredient('heat',['wheat']))

def get_cooking_method(word):
    methods1 = ['[Ss]tand','[Ww]ait','[Bb]oil.*','[Ss]aut[eé]','[Bb]ak(e|ing)','[Ff]r(y|ied)','[Rr]oast', '[Gg]rill','[Ss]team','[Pp]oach','[Ss]immer','[Bb]roil','[Bb]lanch','brais(e|ing)','[Ss]tew']
    methods2 =['[Re]move','[Bb]roil','[Bb]rais(e|ing)','[Cc]hop','[Dd]ic(e|ing)','[Mm]inc(e|ing)','[Mm]uddl(e|ing)','[Ss]i[nm]mer','[G]rat(e|ing)','[Ss]tir','[Ss]hak(e|ing)','[Cc]rush','[Ss]queez(e|ing)','[Cc]ook','[Rr]educ(e|ing)','[Dd]rain','[Mm]ix','[Tt]op','[Ss]prinkl(e|ing)','[Cc]ombin(e|ing)','[Ss]pread','[Ss]ear','[Bb]rown','[Cc]har','[Rr]ub','[Cc]hill','[Rr]inc(e|ing)','[Dd]ip','[Hh]eat','[Cc]over']
    for method in methods1:
        if re.search(method,word):
            return True
        
    for method in methods2:
        if re.search(method,word):
            return True
    return False
result = get_cooking_method("chopped")
print(result)

def get_cooking_tools(word):
    tools1 = ['[Pp]ot','[Kk]nife','[Pp]an.?','[Kk]nives','[Gg]rater','[Bb]oard','[Oo]pener','[Cc]up.?','[Ss]poon.?','[Bb]owl.?','[Cc]olander.?','[Pp]eeler.?','[Mm]asher.?','[Ww]hisk.?','[Ss]pinner.?','[Gg]rater.?','[Ss]hear.?','[Jj]uicer','[Pp]ress','[Ss]teel','[Ss]harpener.?']
    tools2 = ['[Ff]oil','skillet','plate.?','[Ss]patula.?','[Ss]poon.?','[Tt]ong.?','[Ll]adle.?','[Mm]itt.?','[Oo]ven','[Tt]rivet.?','[Gg]uard','[Tt]hermometer.?','[Bb]lender.?','[Ss]cale.?','[Cc]ontainer.?']
#     print(tools[2])
#     print(re.search(tools[4],'spoon'))
    for tool in tools1:
        if re.search(tool,word):
            return True
    for tool in tools2:
        if re.search(tool,word):
            return True
    return False
# get_cooking_tools('spoon')

True


In [356]:
import copy
def analyze_sentence(text, ingredients):
    text = text.replace("Watch Now"," ")
    stop_words = set(stopwords.words('english')) 
    tokenized = sent_tokenize(text) 
    steps = {}
    methods = []
    tools = []
    ingredient = []
    time = []
    for i in tokenized: 
        wordsList = nltk.word_tokenize(i) 
        # removing stop words from wordList 
        wordsList = [w for w in wordsList if not w in stop_words]  
        #  tagged = nltk.pos_tag(wordsList) 
        pre_word = ''
        for word in wordsList:
            if get_cooking_method(word):
                methods.append(word)
            
            if get_cooking_tools(word):
                tools.append(word)   
                
            if get_ingredient(word, ingredients)!=False:
                ingredient.append(get_ingredient(word, ingredients))
            
            if word=='minute' or word == 'minutes':
                time.append(pre_word+' '+'minute')
            pre_word = word
                                  
    steps['ingredient'] = ingredient
    steps['cooking tools'] = tools
    steps['cooking methods'] = methods
    steps['time'] = time
    return steps

def refine_step_ingredients(step, ingredients):
    ingredient_names = copy.deepcopy(ingredients) 
    step_ingredients = step['ingredient']
    ingredients = []
    for ingredient in step_ingredients:
        if ingredient in ingredient_names:
            ingredients.append(ingredient)
            ingredient_names.remove(ingredient)
    step['ingredient'] = ingredients
    return step    

def refine_step_methods(step):
    methods1 = ['[Ss]tand','[Ww]ait','[Bb]oil.*','[Ss]aut[eé]','[Bb]ak(e|ing)','[Ff]r(y|ied)','[Rr]oast', '[Gg]rill','[Ss]team','[Pp]oach','[Ss]immer','[Bb]roil','[Bb]lanch','brais(e|ing)','[Ss]tew']
    methods2 =['[Re]move','[Bb]roil','[Bb]rais(e|ing)','[Cc]hop','[Dd]ic(e|ing)','[Mm]inc(e|ing)','[Mm]uddl(e|ing)','[Ss]i[nm]mer','[G]rat(e|ing)','[Ss]tir','[Ss]hak(e|ing)','[Cc]rush','[Ss]queez(e|ing)','[Cc]ook','[Rr]educ(e|ing)','[Dd]rain','[Mm]ix','[Tt]op','[Ss]prinkl(e|ing)','[Cc]ombin(e|ing)','[Ss]pread','[Ss]ear','[Bb]rown','[Cc]har','[Rr]ub','[Cc]hill','[Rr]inc(e|ing)','[Dd]ip','[Hh]eat','[Cc]over']

    step_methods = step['cooking methods'] 
    methods = []
    for method in step_methods:
        for m in methods1:
            flag = False
            if re.search(m, method):
                methods.append(method)
                methods1.remove(m)
                flag = True
                break
            if flag:
                continue
        for m in methods2:
            if re.search(m, method):
                methods.append(method)
                methods2.remove(m)
                break
    step['cooking methods'] = methods
    return step

def refine_step_tools(step):
    tools1 = ['[Pp]ot','[Kk]nife','[Pp]an.?','[Kk]nives','[Gg]rater','[Bb]oard','[Oo]pener','[Cc]up.?','[Ss]poon.?','[Bb]owl.?','[Cc]olander.?','[Pp]eeler.?','[Mm]asher.?','[Ww]hisk.?','[Ss]pinner.?','[Gg]rater.?','[Ss]hear.?','[Jj]uicer','[Pp]ress','[Ss]teel','[Ss]harpener.?']
    tools2 = ['[Ff]oil','skillet','plate.?','[Ss]patula.?','[Ss]poon.?','[Tt]ong.?','[Ll]adle.?','[Mm]itt.?','[Oo]ven','[Tt]rivet.?','[Gg]uard','[Tt]hermometer.?','[Bb]lender.?','[Ss]cale.?','[Cc]ontainer.?']

    step_tools = step['cooking tools'] 
    tools = []
    for tool in step_tools:
        for t in tools1:
            flag = False
            if re.search(t, tool):
                tools.append(tool)
                tools1.remove(t)
                flag = True
                break
            if flag:
                continue
        for t in tools2:
            if re.search(t, tool):
                tools.append(tool)
                tools2.remove(t)
                break
    step['cooking tools'] = tools
    return step
                
def get_tools_and_methods(directions):
    methods1 = ['[Ss]tand','[Ww]ait','[Bb]oil.*','[Ss]aut[eé]','[Bb]ak(e|ing)','[Ff]r(y|ied)','[Rr]oast', '[Gg]rill','[Ss]team','[Pp]oach','[Ss]immer','[Bb]roil','[Bb]lanch','brais(e|ing)','[Ss]tew']
    methods2 =['[Re]move','[Bb]roil','[Bb]rais(e|ing)','[Cc]hop','[Dd]ic(e|ing)','[Mm]inc(e|ing)','[Mm]uddl(e|ing)','[Ss]i[nm]mer','[G]rat(e|ing)','[Ss]tir','[Ss]hak(e|ing)','[Cc]rush','[Ss]queez(e|ing)','[Cc]ook','[Rr]educ(e|ing)','[Dd]rain','[Mm]ix','[Tt]op','[Ss]prinkl(e|ing)','[Cc]ombin(e|ing)','[Ss]pread','[Ss]ear','[Bb]rown','[Cc]har','[Rr]ub','[Cc]hill','[Rr]inc(e|ing)','[Dd]ip','[Hh]eat','[Cc]over']
    tools1 = ['[Pp]ot','[Kk]nife','[Pp]an.?','[Kk]nives','[Gg]rater','[Bb]oard','[Oo]pener','[Cc]up.?','[Ss]poon.?','[Bb]owl.?','[Cc]olander.?','[Pp]eeler.?','[Mm]asher.?','[Ww]hisk.?','[Ss]pinner.?','[Gg]rater.?','[Ss]hear.?','[Jj]uicer','[Pp]ress','[Ss]teel','[Ss]harpener.?']
    tools2 = ['[Ff]oil','skillet','plate.?','[Ss]patula.?','[Ss]poon.?','[Tt]ong.?','[Ll]adle.?','[Mm]itt.?','[Oo]ven','[Tt]rivet.?','[Gg]uard','[Tt]hermometer.?','[Bb]lender.?','[Ss]cale.?','[Cc]ontainer.?']
    
    methods = []
    tools = []
    stop_words = set(stopwords.words('english')) 
    for direction in directions:
        
        tokenized = sent_tokenize(direction) 
        
        for i in tokenized: 
            wordsList = nltk.word_tokenize(i) 
            wordsList = [w for w in wordsList if not w in stop_words]  
            for word in wordsList:
                for m in methods1:
                    flag = False
                    if re.search(m, word):
                        methods.append(word)
                        methods1.remove(m)
                        flag = True
                        break
                    if flag:
                        continue
                for m in methods2:
                    if re.search(m, word):
                        methods.append(word)
                        methods2.remove(m)
                        break
                
                for t in tools1:
                    flag = False
                    if re.search(t, word):
                        tools.append(word)
                        tools1.remove(t)
                        flag = True
                        break
                    if flag:
                        continue
                for t in tools2:
                    if re.search(t, word):
                        tools.append(word)
                        tools2.remove(t)
                        break
        
    return methods, tools
             

    
def get_steps(directions, ingredients):
    
    steps = []
    tools = []
    methods = []
    for direction in directions:
        sentences = re.split(r"\.|;", direction)
        for s in sentences:
            if s.strip()=='Watch Now' or s.strip()=='':
                continue
#             print(s)
            step = analyze_sentence(s,ingredients)
            step = refine_step_ingredients(step,ingredients)
            step = refine_step_methods(step)
            step = refine_step_tools(step)
            steps.append([s,step])
#             print(step)
#             print()
    
    methods,tools = get_tools_and_methods(directions)
    return tools, methods, steps
    
            

            

    
text = ['Preheat oven to 350 degrees F (175 degrees C).\n                                    Watch Now',
  'Fill a large pot with lightly salted water and bring to a rolling boil over high heat. Once the water is boiling, add the lasagna noodles a few at a time, and return to a boil. Cook the pasta uncovered, stirring occasionally, until the pasta has cooked through, but is still firm to the bite, about 10 minutes. Remove the noodles to a plate.\n                                    Watch Now',
  'Place the ground beef into a skillet over medium heat, add the garlic, garlic powder, oregano, salt, and black pepper to the skillet. Cook the meat, chopping it into small chunks as it cooks, until no longer pink, about 10 minutes. Drain excess grease.\n                                    Watch Now',
  'In a bowl, mix the cottage cheese, eggs, and Parmesan cheese until thoroughly combined.\n                                    Watch Now',
  'Place 4 noodles side by side into the bottom of a 9x13-inch baking pan; top with a layer of the tomato-basil sauce, a layer of ground beef mixture, and a layer of the cottage cheese mixture. Repeat layers twice more, ending with a layer of sauce; sprinkle top with the mozzarella cheese. Cover the dish with aluminum foil.\n                                    Watch Now',
  'Bake in the preheated oven until the casserole is bubbling and the cheese has melted, about 30 minutes. Remove foil and bake until cheese has begun to brown, about 10 more minutes. Allow to stand at least 10 minutes before serving.\n                                    Watch Now']
# text = ['Once the water is boiling, add the lasagna noodles a few at a time, and return to a boil','In a bowl, mix the cottage cheese, eggs, and Parmesan cheese until thoroughly combined.']

# text = ['Heat olive oil in a large saucepan over medium-high heat; saute bacon, onion, garlic, and red pepper flakes in hot oil until onion is golden brown, 10 to 15 minutes. Stir white wine into bacon mixture; cook and stir until wine is absorbed, 2 to 4 minutes. ']
# get_steps(text,ingredients=[])
methods, tools = get_tools_and_methods(text)
print(methods)
print(tools)
# analyze_sentence(text[0])
# print(text[0])

['Preheat', 'boil', 'Cook', 'uncovered', 'stirring', 'Remove', 'chopping', 'Drain', 'mix', 'combined', 'baking', 'top', 'sprinkle', 'brown', 'stand']
['oven', 'pot', 'plate', 'skillet', 'bowl', 'pan', 'foil']


In [358]:
import script
keywords = 'meat lasagna'
rf = script.RecipeFetcher()
meat_lasagna = rf.search_recipes(keywords)[0]
result = rf.scrape_recipe(meat_lasagna)
# print(result)
# print(result['ingredients'])
# print(result['directions'])
ings = result['ingredients']
ingredients = []
for ingredient in ings:
    ingredients.append(tokenize_ingredient(ingredient))

# getMeat("meat")
ingredient_names = []
for ingredient in ingredients:
    ingredient_names.append(ingredient['ingredient_name'])
#     print(ingredient['ingredient_name'])
directions = result['directions']
tools, methods, steps = get_steps(directions, ingredient_names)

print(tools)
print(methods)
for step in steps:
    print(step[0])
    print(step[1])
    print()
    
    

['oven', 'pot', 'plate', 'skillet', 'bowl', 'pan', 'foil']
['Preheat', 'boil', 'Cook', 'uncovered', 'stirring', 'Remove', 'chopping', 'Drain', 'mix', 'combined', 'baking', 'top', 'sprinkle', 'brown', 'stand']
Preheat oven to 350 degrees F (175 degrees C)
{'ingredient': [], 'cooking tools': ['oven'], 'cooking methods': ['Preheat'], 'time': []}

Fill a large pot with lightly salted water and bring to a rolling boil over high heat
{'ingredient': ['whole wheat lasagna noodles '], 'cooking tools': ['pot'], 'cooking methods': ['boil', 'heat'], 'time': []}

 Once the water is boiling, add the lasagna noodles a few at a time, and return to a boil
{'ingredient': ['whole wheat lasagna noodles '], 'cooking tools': [], 'cooking methods': ['boiling'], 'time': []}

 Cook the pasta uncovered, stirring occasionally, until the pasta has cooked through, but is still firm to the bite, about 10 minutes
{'ingredient': ['tomato-basil pasta sauce '], 'cooking tools': [], 'cooking methods': ['Cook', 'uncovere

In [355]:
import re

s = "You[60 pha].Customer(16 ounce) ..."
m = re.search(r"\(([0-9]* [A-Za-z0-9_]+)\)", s)
print(m.group(1))
# def tokenize(text):
#     tokenized = sent_tokenize(text) 
#     for i in tokenized: 
#         wordsList = nltk.word_tokenize(i) 

#         tagged = nltk.pos_tag(wordsList) 

#         print(tagged) 

# tokenize('12 whole wheat lasagna noodles')
# tokenize(' 1 pound lean ground beef')
# tokenize(' 1 (16 ounce) package cottage cheese')
tools = ['pan']
print(re.search(tools[0],'pasta'))

import re
mystr = "ajflkjaldfjsfkj; ajdkfjalkdjfksajf.ksdaljflkjalkfj"
re.split(r"\.|;", mystr)

import Levenshtein as lev
def leven(str1, str2):
    Distance = lev.distance(str1.lower(), str2.lower())
    Ratio = lev.ratio(str1.lower(), str2.lower())
    print(Ratio)
    if Ratio > .95:
        return True
    else:
        return False
leven('boil','boiling')

16 ounce
None
0.7272727272727273


False